# Normalize to 500 KM using MSIS





In [1]:
import pandas as pd
import numpy  as np

from pygeodyn.pygeodyn_plot_scalingfactors import *
import os


### Load GFO

In [2]:
month_list = ['oct', 'nov', 'dec', 'jan', 'feb', 'mar', 'apr']
# month_list = ['nov']

# gfo_file = 'gfo_nov.csv'
# ice_file = 'icesat2_nov.csv'
gfo_file = 'gfo_6month.csv'
ice_file = 'icesat2_6month.csv'
scale_cadence=3

In [3]:
if os.path.exists(gfo_file) :
    print(gfo_file, ' exists' )

else:
    gfo_bigdf = {}

    for imonth,month in enumerate(month_list):
        if month=='oct':
            m_num = 10
            y_num = 2018
        if month=='nov':
            m_num = 11
            y_num = 2018
        if month=='dec':
            m_num = 12
            y_num = 2018
        if month=='jan':
            m_num = 1
            y_num = 2019
        if month=='feb':
            m_num = 2
            y_num = 2019
        if month=='mar':
            m_num = 3
            y_num = 2019
        if month=='apr':
            m_num = 4
            y_num = 2019

        path_gfo     = "/data/SatDragModelValidation/data/inputs/raw_inputdata/data_GRACEFO/"
        filename_gfo = f'GC_DNS_ACC_{y_num}_{m_num:02d}_v02.txt'

        datapath_gfo     = path_gfo + filename_gfo

        headers = [
            'date',        #         Date (yyyy-mm-dd)
            'time',        #         Time (hh:mm:ss.sss)
            'time_system', #         Time system: UTC or GPS (differs per mission)
            'alt',         #  f10.3  Altitude (m), GRS80
            'lon',         #   f8.3  Geodetic longitude (deg), GRS80
            'lat',         #   f7.3  Geodetic latitude (deg), GRS80
            'lst',         #   f6.3  Local solar time (h)
            'arglat',      #   f7.3  Argument of latitude (deg)
            'dens_x',      #  e15.8  Density derived from accelerometer measurements (kg/m3)
            'dens_mean',   #  e15.8  Running orbit average of density (kg/m3)
            'flag_den',    #   f4.1  Flag for density: 0 = nominal data, 1 = anomalous data (-)
            'flag_orbitavg',#   f4.1  Flag for running orbit average density: 0 = nominal data, 1 = anomalous data (-)
                    ]


        gfo_bigdf[month] = pd.read_csv(datapath_gfo, 
                skiprows = 38, 
                sep = '\s+',
                names = headers,
                           )

        #Convert date from GPS to UTC
        date = pd.to_datetime(\
                            + gfo_bigdf[month]['date']  \
                            + gfo_bigdf[month]['time'], \
                                    format='%Y-%m-%d%H:%M:%S.000') - pd.to_timedelta(18,'s')

        gfo_bigdf[month].insert(0, 'Date', date)

        del gfo_bigdf[month]['date'], gfo_bigdf[month]['time'], date
        del gfo_bigdf[month]['time_system']
        del gfo_bigdf[month]['dens_mean']
        del gfo_bigdf[month]['flag_den']
        del gfo_bigdf[month]['flag_orbitavg']


    #     resid_meas_summry = pd.concat([ gfo_bigdf, resid_meas_summry_iter])

    gfo_df = pd.concat([ gfo_bigdf[month] for month in month_list]  )
    gfo_df = gfo_df.reset_index(drop=True)


gfo_6month.csv  exists


### Load Icesat2

In [4]:
if os.path.exists(ice_file):
    print(ice_file, ' exists' )
    
else:


    %load_ext autoreload
    %autoreload 2
    from gc import collect as gc_collect
    import pickle 
    from datetime import datetime,timedelta

    run_list = ['msis2',
                'jb2008',
                'dtm2020_o']

#     month_list = ['nov']


    scale_cadence = 3

    dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
    run_dict={}
    for imonth,month in enumerate(month_list):
        for i in run_list:
            if i =='msis2':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 5
                run_dict[month+i]['model_path'] = None
            if i =='dtm2020_o':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 3
                run_dict[month+i]['model_path'] = None
            if i =='jb2008':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 1
                run_dict[month+i]['model_path'] = None

    print(run_dict)



    dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
                 + 'icesat2/O2R2023_longimeperiod/1_DRIAruns/'
    obj = {}
    for imonth,month in enumerate(month_list):
        for i,model in enumerate(run_list):
            pickleName = f'_{month}_DRIA_scale{scale_cadence}.pkl'

            ### Load the data if the pickles exist
            print()
            print()
            gc_collect()

            pickle_file = dir_save+model+pickleName

            filehandler = open(pickle_file, 'rb') 
            obj[month+model] = pickle.load(filehandler)
            filehandler.close()
            print('Loaded data from pickle... ',  month+model)


    ### Save space if doing density retrieval
    for model in run_dict.keys():
        del obj[model]['OrbitResids']
        del obj[model]['Trajectory_orbfil']

    gc_collect()
    
    
    ## MAKE SCALING FACTORS
    
    satid = 1807001
    wgts = {}

    for model in run_dict.keys():
        wgts[model] = {}
        ScalingFactors  = []
        ScalingFactor_times = []

        for ii,arc in enumerate(obj[model]['global_params']['arc_input']):
            epochstart = obj[model]['global_params']['prms']['epoch_start'][ii]
            hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
            frachours =(hrs/24)
            #
            if len(arc) == 9:
                maneuv_indicator = arc[8]
            else:
                maneuv_indicator = ''
            arc_type = obj[model]['global_params']['prms']['arc_type']
            if arc_type == "Nominal30hr_and_AB":
                arc_name =arc[:8]+ maneuv_indicator
            else:
                arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
            ### Collect the weights for the ensemble average
            inv_rms          = 1/obj[model]['Statistics'][arc_name]['T_RMS'].values[0]
            wgts[model][arc_name] = inv_rms#/sum_wgts

            iters = int(obj[model]['run_parameters'+arc_name]['total_iterations']) 
            for iit, itime in enumerate(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):
                if iit == 0 or iit==9:
                    pass
                else:
                    CURRENT_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['CURRENT_VALUE']
                    APRIORI_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']
                    ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
                    ScalingFactor_times.append(itime)
        run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
        run_dict[model]['ScalingFactors']      = ScalingFactors
        run_dict[model]['Weight'] = wgts[model]



    ###### Scale the densities
    models_dens = {}
    for monthmodel in run_dict.keys():
        print(f"---Making continuous scaled rho for {monthmodel}")
        models_dens[monthmodel] = get_continuous_scaled_densities(obj, run_dict, monthmodel, scale_cadence)


    ## Retrieve scaled ensemble weighted average
    ##     'Rho_x' denotes the ensemble weighted avg
    print(f"---Making ensemble avg")
#     for monthmodel in run_dict.keys():
#         month = monthmodel[:3]
    models_dens =  calc_rho_ScaledEnsembleWgtAvg(models_dens, run_dict, month_list, run_list)


    ### Clear up space 
    del obj
    for monthmodel in run_dict.keys():
        print(monthmodel)
        if 'Rho_x' not in monthmodel:
            del models_dens[monthmodel]

    gc_collect()       

icesat2_6month.csv  exists


## Process normalization

In [5]:
%load_ext autoreload
%autoreload 2
from pygeodyn.pygeodyn_plot_scalingfactors import *

In [6]:

if os.path.exists(gfo_file) :
    print(gfo_file, ' exists' )
    
    
    gfo_df = pd.read_csv(gfo_file, 
                    sep = ',',
                    )

    
    
else:
    print(f"---Calculating Grace-FO normalization")
    D500_gfo = normalize_density_msis2( gfo_df , 'GRACE-FO', 500)
    gfo_df['D500_gfo'] = D500_gfo
    #### save to a csv
    gfo_df.to_csv(gfo_file, index=False)  

    

# gfo_df2 =     gfo_df.query("Date >= '2018-10-14' and Date < '2018-12-30'")
# del gfo_df

gfo_6month.csv  exists


In [7]:
if os.path.exists(ice_file) :
    print(ice_file, ' exists' )
    
    ice_df = pd.read_csv(ice_file, 
                    sep = ',',
                    )

    
else:
    
    ice_bigdf = {}
    print(f"---Calculating ICESat-2 normalization")
    
    for imonth,month in enumerate(month_list):

        D500_ice = normalize_density_msis2( models_dens[month+'Rho_x'], 'ICESat-2', 500)
        models_dens[month+'Rho_x']['D500_ice'] = D500_ice
        
        ice_bigdf[month] = pd.DataFrame.from_dict(models_dens[month+'Rho_x'])
#     ice_df.to_csv('icecat2_3month.csv', index=False)  

    
    ice_df = pd.concat([ ice_bigdf[month] for month in month_list]  )
    ice_df = ice_df.reset_index(drop=True)
    ice_df.to_csv(ice_file, index=False)  

    ### Need to set this up to only write to file once. and have that write include the full dataset. Maybe concat the dataframes by month first



icesat2_6month.csv  exists


In [8]:
# gfo_df['Date']


In [9]:
# ice_df['datescaled']

In [10]:
(timeavg_gfo,  denavg_gfo) = orbit_avg_generic(gfo_df['Date'], gfo_df['D500_gfo'], gfo_df['lat'])    
(timeavg_ice,  denavg_ice) = orbit_avg_generic(ice_df['date'], ice_df['D500_ice'], ice_df['lat'])    


# remove the datapoints that are definitely artifacts of the orbit average
# 74, 334,  365  

# timeavg_ice.pop(74)
# timeavg_ice.pop(334)
# timeavg_ice.pop(365)
# list(denavg_ice).pop(74)
# list(denavg_ice).pop(334)
# list(denavg_ice).pop(365)


# timeavg_ice[74]=np.nan
# timeavg_ice[334]=np.nan
# timeavg_ice[365]=np.nan
# denavg_ice[74]=np.nan
# denavg_ice[334]=np.nan
# denavg_ice[365]=np.nan


For every time in the GRACEFO orbit average, I have found the closest time in the ICESat-2 orbit average and percent differenced those values

In [11]:

def calc_percent_diff_logspace(model, obs):
    from numpy import exp as np_exp
    from numpy import log as np_log # this is the natural log
    
    return( ( np_exp(np_log(model/obs))-1 )*100)


In [12]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))


# def calc_percent_change(a, b):
#     from numpy import absolute as np_abs
#     return(  ( (b-a)/np_abs(a) )*100)

# def calc_percent_diff(a, b):
#     return(  ( (b-a)/((a+b)/2) )*100)





perc_stat = {}
perc_stat['time_midpoint'] = []
perc_stat['gfo_denorbavg'] = []
perc_stat['ice_denorbavg'] = []
perc_stat['gfo_time']      = []
perc_stat['ice_time']      = []
# perc_stat['percchange']    = []
perc_stat['percdiff_log']    = []
# perc_stat['percdiff_log2']    = []
perc_stat['ratio']    = []


from numpy import exp as np_exp
from numpy import log as np_log # this is the natural log


for i,val in enumerate( timeavg_gfo):
    date_near = nearest(timeavg_ice, val)
    indx = np.where(pd.to_datetime(timeavg_ice) == pd.to_datetime(date_near))[0][0]
    
    if np.abs((val - timeavg_ice[indx]).total_seconds()) > 5700:  # only compute % change if values within 1 orbit (95mins)
        pass
    
    else:
        if np.abs((timeavg_ice[indx] - timeavg_ice[indx-1]).total_seconds()) > 40000:
            print('skipping ', indx, timeavg_ice[indx])
        else:
      

            perc_stat['time_midpoint'].append( pd.Timestamp(val) + (pd.Timestamp(timeavg_ice[indx]) - pd.Timestamp(val)) / 2)
            perc_stat['gfo_denorbavg'].append( denavg_gfo[i])
            perc_stat['ice_denorbavg'].append( denavg_ice[indx])
            perc_stat['gfo_time'].append( val)
            perc_stat['ice_time'].append( timeavg_ice[indx])
#             perc_stat['percchange'].append( calc_percent_change(denavg_gfo[i], denavg_ice[indx])  )
            perc_stat['percdiff_log'].append(calc_percent_diff_logspace(denavg_ice[indx], denavg_gfo[i]) )
            perc_stat['ratio'].append(denavg_ice[indx]/denavg_gfo[i])
    
    

#     print(val, date_near)
    
    
#  2018-10-17 00:22:04
#  2018-10-29 00:32:49
#  2018-11-07 00:02:14
#  2018-11-25 11:58:44
#  2018-11-29 00:03:14
#  2018-12-06 00:12:14
#  2018-12-09 11:29:39
#  2018-12-13 00:21:14
#  2018-12-20 12:17:29
#  2018-12-27 23:26:34
#  2018-12-31 12:18:14
#  2019-01-12 12:35:09
#  2019-01-17 00:43:49
#  2019-01-23 00:06:44
#  2019-02-12 00:40:24
#  2019-02-20 00:32:04
#  2019-03-12 00:01:09
#  2019-03-28 00:41:44
#  2019-04-02 00:24:44
#  2019-04-17 00:20:24  


skipping  0 2018-10-14 00:51:54
skipping  0 2018-10-14 00:51:54
skipping  28 2018-10-17 00:22:04
skipping  28 2018-10-17 00:22:04
skipping  29 2018-10-18 02:17:54
skipping  181 2018-10-29 00:32:49
skipping  181 2018-10-29 00:32:49
skipping  182 2018-10-30 01:52:54
skipping  182 2018-10-30 01:52:54
skipping  288 2018-11-07 00:02:14
skipping  288 2018-11-07 00:02:14
skipping  289 2018-11-08 01:10:49
skipping  289 2018-11-08 01:10:49
skipping  548 2018-11-25 11:58:44
skipping  548 2018-11-25 11:58:44
skipping  549 2018-11-26 01:20:14
skipping  549 2018-11-26 01:20:14
skipping  579 2018-11-29 00:03:14
skipping  579 2018-11-29 00:03:14
skipping  580 2018-11-30 01:11:54
skipping  580 2018-11-30 01:11:54
skipping  656 2018-12-06 00:12:14
skipping  656 2018-12-06 00:12:14
skipping  657 2018-12-07 01:20:54
skipping  671 2018-12-09 11:29:39
skipping  671 2018-12-09 11:29:39
skipping  672 2018-12-11 01:12:34
skipping  686 2018-12-13 00:21:14
skipping  686 2018-12-13 00:21:14
skipping  687 2018-12

In [13]:
%load_ext autoreload
%autoreload 2

from pygeodyn.pygeodyn_plot_scalingfactors import *
import os
import pandas as pd
from netCDF4 import Dataset
def read_nc_file( filename, variables):
    ''' This function reads the TIEGCM .nc files and saves the given input variables to a dictionary.
        The breakloop feature is here so that if the file doesn't exist the code can still continue.  '''
    status = os.path.exists(filename)
    
    if status == True:
        data = {}
        for i, var_names in enumerate(variables):
            ncid =  Dataset(filename,"r+", format="NETCDF4")# filename must be a string
            varData = ncid.variables
            data[var_names] = np.array(varData[var_names])  
    elif status == False:
        print('No File Found', filename )
        breakloop = True
        data = 0
        return( data , breakloop)
    breakloop = False
    return(data,breakloop )


arc_list = []

arc_list_18 = np.arange(287,365)
# arc_list_18 = np.arange(304,335)
for i in arc_list_18:
    val = '2018'+str(i)
    arc_list.append(int(val))
    
    #     print(val)
    
arc_list_19 = np.arange(1,110)
for i in arc_list_19:
    val = f"2019{i:03d}"
    arc_list.append(int(val))

path_to_f107 = '/data/SatDragModelValidation/data/inputs/atmos_models/geo_phys_indicies/gpi_1960001-2021243_f107aDaily.nc'

f107_data = read_nc_file(path_to_f107, ['year_day', 'f107d', 'f107a', 'kp'])


date = []
kp_list = []
f107d_list = []
f107a_list  = []
date_3hr = []
doy_list    = []



for i,val in enumerate(arc_list):
    
    index = f107_data[0]['year_day']==val
    kp_list.append(f107_data[0]['kp'][index][0])
    f107d_list.append(f107_data[0]['f107d'][index][0])
    f107a_list.append(f107_data[0]['f107a'][index][0])
    doy_list.append(str(f107_data[0]['year_day'][index][0])[-3:])

    date.append(pd.to_datetime( str(val), format='%Y%j'))

    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=0))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=3))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=6))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=9))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=12))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=15))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=18))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=21))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=24))
    
kp_expand = []
for i in kp_list:
    for ii in i:
        kp_expand.append(ii)
        
        
        
solar_fluxes = {}
solar_fluxes['f107d_list'] = f107d_list
solar_fluxes['f107a_list'] = f107a_list
solar_fluxes['date']       = date
solar_fluxes['date_3hr']   = date_3hr
solar_fluxes['kp_expand']  = kp_expand

f107d_earth = []
f107a_earth = []
######################################################################### 
##### Account for the F10.7 at earth (instead of referenced at 1AU) #####
######################################################################### 

for i_doy,val_doy in enumerate(doy_list):
    iday = int(val_doy)
    theta0 = 2 * np.pi * (iday)/365.
    sfeps = 1.000110 + 0.034221*np.cos(theta0)+0.001280* np.sin(theta0) +0.000719*np.cos(2.*theta0)+0.000077*np.sin(2.*theta0)

    f107d_earth.append(sfeps * solar_fluxes['f107d_list'][i_doy])
    f107a_earth.append(sfeps * solar_fluxes['f107a_list'][i_doy])

solar_fluxes['f107d_earth'] = f107d_earth
solar_fluxes['f107a_earth'] = f107a_earth

 
del f107d_earth
del f107a_earth
del kp_expand
del f107d_list
del f107a_list
del date
del date_3hr
del f107_data





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:

# fig = make_subplots(rows=1, cols=1)
fig = make_subplots(rows=3, cols=1, row_heights=[0.2, 0.4, 0.3],#,0.2],
                    specs=[[{"secondary_y": True}],
                           [{"secondary_y": False}],
#                            [{"secondary_y": False}],
                           [{"secondary_y": False}]],
                           shared_xaxes=True,
                           vertical_spacing=0.02)

fig.add_trace(go.Scatter(x=solar_fluxes['date'],
                           y=solar_fluxes['f107d_earth'],
                           name= 'F107d_1AU',
                           mode='lines',
                           opacity=1,
                           line = dict(shape = 'hvh',dash='dash', color = 'blue', width=2),
                           showlegend=False),
                           secondary_y=True,row=1, col=1)

fig.add_trace(go.Scatter(x=solar_fluxes['date_3hr'],
                           y=solar_fluxes['kp_expand'],
                           name= 'Kp',
                           mode='lines',
                           opacity=1,
                           line = dict(shape = 'hvh', color = 'black', width=2),
                           showlegend=False),
                           secondary_y=False,row=1, col=1) 



fig.add_trace(go.Scattergl(x= perc_stat['gfo_time'],
                            y=perc_stat['gfo_denorbavg'],
                            name=f'GFO_500km',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=4, color="#d62728" ),
                            showlegend=False),
                            row=2, col=1)

fig.add_trace(go.Scattergl(x=perc_stat['ice_time'],
                            y=perc_stat['ice_denorbavg'],
                            name=f'D500_icesat2',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=4, color="#1f77b4"),
                            showlegend=False),
                            row=2, col=1)



fig.add_trace(go.Scattergl(x=perc_stat['time_midpoint'],
                            y=perc_stat['percdiff_log'],
#                             name=f'D500_icesat2',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=4, color='black'),
                            showlegend=False),
                            row=3, col=1)
# window_size = 50
# i = 0
# # Initialize an empty list to store moving averages
# moving_averages = []
# # Loop through the array to consider
# # every window of size 3
# while i < len(perc_stat['percdiff_log']) - window_size + 1:
#     # Store elements from i to i+window_size
#     # in list to get the current window
#     window = perc_stat['percdiff_log'][i : i + window_size]
#     # Calculate the average of current window
#     window_average = round(sum(window) / window_size, 2)
#     # Store the average of current
#     # window in moving average list
#     moving_averages.append(window_average)
#     # Shift window to right by one position
#     i += 1
fig.add_hline(y=0, line = dict(dash='solid', color = 'grey', width=1), row=3, col=1)
# fig.add_trace(go.Scattergl(x=perc_stat['time_midpoint'],
#                             y=moving_averages,
#                             mode='markers+lines',
#                             opacity=1,
#                             marker=dict( size=2, color="#ff7f0e"),
#                             showlegend=False),
#                             row=3, col=1)
N = len(perc_stat['ratio'])
sum_log = 0
sum_log2 = 0
for i in range(N):
    sum_log  += np_log(perc_stat['ratio'][i])
    sum_log2 += np_log(perc_stat['ratio'][i])**2
#         np_exp((1/N)*perc_stat['percdiff_log2'])-1 )*100
per_diff_log     = 100* (np_exp( (1/N)*sum_log  )-1)
per_rms_diff_log = 100* (np_exp( np.sqrt((1/N)*(sum_log2))  )-1)
print(f"% difference in log space:     % {per_diff_log:6.3f}")
print(f"% RMS difference in log space: % {per_rms_diff_log:6.3f}")

# fig.add_annotation(
#                 y= .28,
#                 x= .8,
#                 xref="paper", yref="paper",
#                 showarrow=False,
#                 # text="dZ Night = "+str(np.round(deltaZ_night,2))+" km", 
#                 text=f"% diff: {str(np.round(per_diff_log,2))}%", 
#                 align = 'left',
#                 valign = 'top',
# #                 bgcolor='lightgrey',
# #                 bordercolor="blue",
#                 borderwidth=1,
#                 borderpad=2,
#                 font=dict(family='Arial',size=18,color="black"),
#                 )
fig.add_annotation(
#                 y= .025,
                y= .25,
                x= .92,
                xref="paper", yref="paper",
                showarrow=False,
                text=f"% diff: {per_diff_log:05.3f}% <br>% rms diff: {per_rms_diff_log:05.3f}%", 
                align = 'left',
                valign = 'bottom',
                bgcolor='lightgrey',
                bordercolor='black',
                borderwidth=1,
                borderpad=2,
                font=dict(family='Arial',size=14,color="black"),
                )


print(np.mean(perc_stat['percdiff_log']))



#### FANCY LEGEND ################################################################
modelnames=[]
modelcolors = []
modelnames.append("GraceFO")
modelcolors.append("#d62728")
modelnames.append("ICESat2")
modelcolors.append('#1f77b4')
df_leg = pd.DataFrame({"starts_colors": modelcolors})
fig.update_traces(showlegend=False).add_traces(
    [   go.Scattergl(name=modelnames[i],
               x=[pd.to_datetime( "181107-000000", format='%y%m%d-%H%M%S')],
               mode='lines',
               line = dict(shape = 'hv',  width=10),
               marker_color=c,
               showlegend=True)
        for i,c in enumerate((df_leg.loc[:,["starts_colors"]].values.ravel()))])
## Legend Control
fig.update_layout(legend=dict(
    yanchor="top",
    y=.75,
    xanchor="center",
    x=.45,
    orientation="h",
        font=dict(family='Arial',size=12,color='black'),
        bgcolor="white",
        bordercolor="darkgrey",
        borderwidth=0.5,)  )
################################################################################


### UPDATE AXES 
fig.update_yaxes(title_text="Kp", 
                 exponentformat= 'power',
                 range=[0,7],
                 secondary_y=False,
                 row=1, col=1)
fig.update_yaxes(title_text="F10.7", 
                 exponentformat= 'power',
                 range=[20,90],
                 secondary_y=True,
                 tickfont=dict(color="blue"),
                 titlefont=dict(color="blue"),
                 row=1, col=1)
################################################################################

### SYLIZE LEGEND 
font_dict=dict(family='Arial',size=11,color='black')
## automate the specification of the axes for subplots
rownum, colnum = fig._get_subplot_rows_columns()
for i in rownum:
    if len(rownum)==1:
        L_ticklabel = False
    else:
        if i < len(rownum):
            L_ticklabel = False
        else:
            L_ticklabel = True
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=L_ticklabel,
#                       tickformat= '%m/%d',
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
#                       tickwidth=2,
#                       ticklen=9,
                      tickcolor='grey',
#                       tick0="2018-11-9" ,
#                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)


# fig.update_xaxes(range=[pd.to_datetime( "181101-000000", format='%y%m%d-%H%M%S'),
#                         pd.to_datetime( "181130-000000", format='%y%m%d-%H%M%S')],row=1, col=1)


fig.update_yaxes(title_text="Density", 
                 type="log", 
                 exponentformat= 'power',row=2, col=1)
fig.update_yaxes(title_text="Percent Difference", 
                minor=dict(dtick=10, tickwidth=1,ticklen=4,tickcolor='grey',ticks='inside'),
                 exponentformat= 'power',row=3, col=1)
# fig.update_yaxes(title_text="Ratio (GFO/ICE)", 
#                 minor=dict(dtick=0.25, tickwidth=1,ticklen=4,tickcolor='grey',ticks='inside'),
#                  exponentformat= 'power',row=4, col=1)

fig.update_layout(margin=dict(l=20, r=20, t=20, b=20),)

fig.update_layout(#title=f"ICESat2 {scale_cadence}-hr Scaled Model Rho vs GRACE-FO Rho, Norm 500km",
                  autosize=False,    width=1000,    height=800,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict, plot_bgcolor='white', 
                 )

fig.show(config=config)

pio.write_image(fig, 'ICEvGFO.jpg', scale=5)


NameError: name 'make_subplots' is not defined

In [15]:
import sys
sys.exit(0)

SystemExit: 0

/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [ ]:
# (timeavg_gfo,  denavg_gfo) = orbit_avg_generic(gfo_df['Date'], gfo_df['D500_gfo'], gfo_df['lat'])    
# (timeavg_ice,  denavg_ice) = orbit_avg_generic(ice_df['date'], ice_df['D500_ice'], ice_df['lat'])    


In [ ]:
gfo_df = gfo_df.query("Date >= '2018-11-10' and Date < '2018-11-13'")
ice_df = ice_df.query("date >= '2018-11-10' and date < '2018-11-13'")



In [ ]:

ice_df

In [ ]:
gfo_df

In [ ]:


# fig = make_subplots(rows=1, cols=1)
fig = make_subplots(rows=1, cols=1,
                           shared_xaxes=True,
                           vertical_spacing=0.02)


fig.add_trace(go.Scattergl(x= gfo_df['Date'],
                            y=gfo_df['D500_gfo'],
                            name=f'GFO_500km',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=4, color="#d62728" ),
                            showlegend=False),
                            row=1, col=1)

fig.add_trace(go.Scattergl(x=ice_df['date'],
                            y=ice_df['D500_ice'],
                            name=f'D500_icesat2',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=4, color="#1f77b4"),
                            showlegend=False),
                            row=1, col=1)











#### FANCY LEGEND ################################################################
modelnames=[]
modelcolors = []
modelnames.append("GraceFO")
modelcolors.append("#d62728")
modelnames.append("ICESat2")
modelcolors.append('#1f77b4')
df_leg = pd.DataFrame({"starts_colors": modelcolors})
fig.update_traces(showlegend=False).add_traces(
    [   go.Scattergl(name=modelnames[i],
               x=[pd.to_datetime( "181111-000000", format='%y%m%d-%H%M%S')],
               mode='lines',
               line = dict(shape = 'hv',  width=10),
               marker_color=c,
               showlegend=True)
        for i,c in enumerate((df_leg.loc[:,["starts_colors"]].values.ravel()))])
## Legend Control
fig.update_layout(legend=dict(
    yanchor="top",
    y=.95,
    xanchor="center",
    x=.55,
    orientation="h",
        font=dict(family='Arial',size=12,color='black'),
        bgcolor="white",
        bordercolor="darkgrey",
        borderwidth=0.5,)  )

################################################################################

### SYLIZE LEGEND 
font_dict=dict(family='Arial',size=11,color='black')
## automate the specification of the axes for subplots
rownum, colnum = fig._get_subplot_rows_columns()
for i in rownum:
    if len(rownum)==1:
        L_ticklabel = True
    else:
        if i < len(rownum):
            L_ticklabel = False
        else:
            L_ticklabel = True
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=L_ticklabel,
#                       tickformat= '%m/%d',
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
#                       tickwidth=2,
#                       ticklen=9,
                      tickcolor='grey',
#                       tick0="2018-11-9" ,
#                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)




fig.update_yaxes(title_text="Density", 
                 type="log", 
                  range=[-13.7-.05,  -12.6+.25],
                 exponentformat= 'power',row=1, col=1)


fig.update_layout(#title=f"ICESat2 {scale_cadence}-hr Scaled Model Rho vs GRACE-FO Rho, Norm 500km",
                  autosize=False,    width=1000,    height=400,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict, plot_bgcolor='white', 
                 )

fig.show(config=config)



In [ ]:

# fig = make_subplots(rows=1, cols=1)




# # fig.add_trace(go.Scattergl(x=gfo_df['Date'][:10000],
# #                             y=gfo_df['D500_gfo'][:10000],
# #                             name=f'D500_gfo',
# #                             mode='markers',
# #                             opacity=1,
# #                             marker=dict( size=4 ),
# #                             showlegend=True),
# #                             row=1, col=1)

# # fig.add_trace(go.Scattergl(x=ice_df['datescaled'],
# #                             y=ice_df['D500_ice'],
# #                             name=f'D500_icesat2',
# #                             mode='markers',
# #                             opacity=1,
# #                             marker=dict( size=4 ),
# #                             showlegend=True),
# #                             row=1, col=1)
# (time_avg, den_gfo ) = orbit_avg_generic(gfo_df['Date'], gfo_df['D500_gfo'], gfo_df['lat'])    
# fig.add_trace(go.Scattergl(x=time_avg,
#                             y=den_gfo,
#                             name=f'D500_gfo',
#                             mode='markers',
#                             opacity=1,
#                             marker=dict( size=4 ),
#                             showlegend=True),
#                             row=1, col=1)

# (time_avg, den_avg ) = orbit_avg_generic(ice_df['datescaled'],ice_df['D500_ice'],ice_df['lat'])    
# # remove the datapoints that are definitely artifacts of the orbit average
# # 74, 334,  365  

# time_avg[74]=np.nan
# time_avg[334]=np.nan
# time_avg[365]=np.nan
# den_avg[74]=np.nan
# den_avg[334]=np.nan
# den_avg[365]=np.nan


# fig.add_trace(go.Scattergl(x=time_avg,
#                             y=den_avg,
#                             name=f'D500_icesat2',
#                             mode='markers',
#                             opacity=1,
#                             marker=dict( size=4 ),
#                             showlegend=True),
#                             row=1, col=1)



# ### SYLIZE AXES 
# font_dict=dict(family='Arial',size=11,color='black')
# ## automate the specification of the axes for subplots
# rownum, colnum = fig._get_subplot_rows_columns()
# for i in rownum:
#     if len(rownum)==1:
#         L_ticklabel = True
#     else:
#         if i < len(rownum):
#             L_ticklabel = True
#         else:
#             L_ticklabel = True
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,
#                       showticklabels=L_ticklabel,
# #                       tickformat= '%m/%d',
#                       linecolor='black',
#                       linewidth=1,
#                      ### Major ticks
#                       ticks='inside',
#                       tickfont=font_dict,
#                       mirror=True,
# #                       tickwidth=2,
# #                       ticklen=9,
#                       tickcolor='grey',
# #                       tick0="2018-11-9" ,
# #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(
#                          dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,
#                          ticklen=4,
#                          tickcolor='grey',
#                          ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',
#                        gridwidth=1,
#                        layer='above traces',
#                        tickangle=0,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=0
#                          showticklabels=True,
#                          linecolor='black',  # line color
#                          linewidth=1,        # line size
#                      ticks='inside',     # ticks outside axis
#                      tickfont=font_dict, # tick label font
#                      mirror='allticks',  # add ticks to top/right axes
#                      tickwidth=1,      # tick width
#                      tickcolor='black',  # tick color
#                      gridcolor='gainsboro',
#                      gridwidth=1,
#                      layer='above traces',
#                      row=i, col=1)




# fig.update_yaxes(title_text="Density", 
#                  type="log", 
#                  exponentformat= 'power',row=1, col=1)
# fig.update_layout(#title="ICESat2 3-hr Scaled Model Rho vs GRACE-FO Rho",
#                   autosize=True,#    width=1000,    height=700,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict, plot_bgcolor='white', 
#                  )

# fig.show(config=config)



In [ ]:
gfo_df

In [ ]:
ice_df

In [ ]:

fig = make_subplots(rows=1, cols=1)




# fig.add_trace(go.Scattergl(x=gfo_df['Date'],
#                             y=gfo_df['D500_gfo'],
#                             name=f'D500_gfo',
#                             mode='markers',
#                             opacity=1,
#                             marker=dict( size=4 ),
#                             showlegend=True),
#                             row=1, col=1)

fig.add_trace(go.Scattergl(x=ice_df['dates'],
                            y=ice_df['D500_ice'],
                            name=f'D500_icesat2',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=4 ),
                            showlegend=True),
                            row=1, col=1)

fig.add_trace(go.Scattergl(x=ice_df['dates'],
                            y=ice_df['Rho_x'],
                            name=f'Rho_x_icesat2',
                            mode='markers',
                            opacity=1,
                            marker=dict( size=4 ),
                            showlegend=True),
                            row=1, col=1)



### SYLIZE AXES 
font_dict=dict(family='Arial',size=11,color='black')
## automate the specification of the axes for subplots
rownum, colnum = fig._get_subplot_rows_columns()
for i in rownum:
    if len(rownum)==1:
        L_ticklabel = True
    else:
        if i < len(rownum):
            L_ticklabel = True
        else:
            L_ticklabel = True
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=L_ticklabel,
#                       tickformat= '%m/%d',
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
#                       tickwidth=2,
#                       ticklen=9,
                      tickcolor='grey',
#                       tick0="2018-11-9" ,
#                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)




fig.update_yaxes(title_text="Density", 
                 type="log", 
                 exponentformat= 'power',row=1, col=1)
fig.update_layout(#title="ICESat2 3-hr Scaled Model Rho vs GRACE-FO Rho",
                  autosize=True,#    width=1000,    height=700,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict, plot_bgcolor='white', 
                 )

fig.show(config=config)



In [ ]:
# import pandas as pd
# import numpy as np 
# import sys  
# from scipy.io import loadmat  #allows us to read in .mat files
# from datetime import datetime, timedelta
# import gc

# #### MAKE MSIS Take the 3HR Ap values
# from pymsis import msis
# SWI_option = [1.0]*25
# SWI_option[8] = -1.0
#             #  C    AP - MAGNETIC INDEX(DAILY) OR WHEN SW(9)=-1. :
#             #  C      - ARRAY CONTAINING:
#             #  C       (1) DAILY AP
#             #  C       (2) 3 HR AP INDEX FOR CURRENT TIME
#             #  C       (3) 3 HR AP INDEX FOR 3 HRS BEFORE CURRENT TIME
#             #  C       (4) 3 HR AP INDEX FOR 6 HRS BEFORE CURRENT TIME
#             #  C       (5) 3 HR AP INDEX FOR 9 HRS BEFORE CURRENT TIME
#             #  C       (6) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 12 TO 33 HRS PRIOR
#             #  C          TO CURRENT TIME
#             #  C       (7) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 36 TO 57 HRS PRIOR
#             #  C          TO CURRENT TIME


# #################################
# years =  [2002]#, 2003, 2004, 2005, 2006, 2007, 2008, 2009]
# days = np.arange(1,2)
# path_champ = '/space/DNR_data/CHAMP_2002_2010/'
# #################################


# def make_champ_timeseries_v2_redorenorm(years, days):

#     """This function redoes the construction of the CHAMP timeseries for the orbit.

#     Changes from the original include:
#         - Use F10.7 indicies that are scaled with MgII and re-referenced to the Earth's locatiion (instead of at 1AU)
#         - Use MSIS2.0 for the normalization to 400km
#         - Use a 3Hour Ap input as required by MSIS for the more granular Stormtime forcing option.




#     """    


#     path_champ = '/space/DNR_data/CHAMP_2002_2010/'


#     #### Load MgII Scaled F10.7 Values:
#     import pickle
#     dir_save = '/space/DNR_data/'
#     filehandler = open(dir_save+'MgII_F107_KpAp'+'.pkl', 'rb') 
#     mgII_data = pickle.load(filehandler)
#     filehandler.close()

#     #### clear up some space        
#     truncate_date    = np.logical_and(mgII_data['Date'].year>=2000 , mgII_data['Date'].year<=2011 )
#     truncate_date3hr =  np.logical_and(mgII_data['Date_3hrAp'].year>=2000 , mgII_data['Date_3hrAp'].year<=2011 )#np.logical_and(mgII_data['Date_3hrAp'].year==year , mgII_data['Date_3hrAp'].dayofyear==day )

#     mgII_data['Date_3hrAp'] = mgII_data['Date_3hrAp'][truncate_date3hr]
#     mgII_data['Ap']         = np.array(mgII_data['Ap'])[truncate_date3hr]
#     mgII_data['Date']       = mgII_data['Date'][truncate_date]
#     mgII_data['Ap_dailyavg'] =  np.array(mgII_data['Ap_dailyavg'])[truncate_date]
#     mgII_data['f107d_earth'] = np.array(mgII_data['f107d_earth'])[truncate_date]
#     mgII_data['f107a_earth'] = np.array(mgII_data['f107a_earth'])[truncate_date]

#     del mgII_data['DOY']
#     del mgII_data['kp']
#     del mgII_data['f107d']
#     del mgII_data['f107a']
#     del mgII_data['year_day']




#     #     noaa = pd.read_pickle('/space/DNR_data/noaa_2002_2010_pickle')
#     #     noaa['f107d'][noaa['f107d'].astype(float) <= 60] = noaa['f107a'][noaa['f107d'].astype(float) <= 60]

#     tleng = 0
#     time_full= []
#     Year         = []
#     Doy          = []
#     Hours        = []
#     Lon          = []
#     Lat          = []
#     LatBin       = []
#     Height       = []
#     LocTim       = []
#     CHAMPDensity = []
#     D400_msis00  = []  # normalized quantity
#     rhosat_msis00= []
#     rhosat_msis2 = np.ones(2000*np.size(years)*np.size(days))*np.nan
#     rho400_msis2 = np.ones(2000*np.size(years)*np.size(days))*np.nan
#     D400_msis2   = np.ones(2000*np.size(years)*np.size(days))*np.nan

#     rhosat_tiegcm = np.ones(2000*np.size(years)*np.size(days))*np.nan
#     rho400_tiegcm = np.ones(2000*np.size(years)*np.size(days))*np.nan
#     D400_tiegcm   = np.ones(2000*np.size(years)*np.size(days))*np.nan

#     #     Cd           = []
#     # Ap_dayvals   = []
#     # f107a_dayvals= []
#     # f107d_dayvals= []
#     date = []

#     i = 0
#     for iyear,year in enumerate(years):
#         for iday,day in enumerate(days):
#                 ####---------------------------------------------
#                 #### Gather the Necessary Flux and Ap Information
#                 index_date = np.logical_and(mgII_data['Date'].year==year , mgII_data['Date'].dayofyear==day )
#                 f107a = float(np.squeeze(np.asarray(mgII_data['f107a_earth'])[index_date]))
#                 f107d = float(np.squeeze(np.asarray(mgII_data['f107d_earth'])[index_date]))
#                 Ap_daily_avg = float(np.squeeze(np.asarray(mgII_data['Ap_dailyavg'])[index_date]))


#                 ### Construct the necessary 3hr Ap Array to go into MSIS
# #                 print(f"----------------------------------------------------------")

#                 print(f"Year: {year} / Day: {day}")
#                 for it, itime in enumerate( champ['Hours'][:leng]):

#                     lon   = champ['Lon'][it]
#                     lat   = champ['Lat'][it]
#                     dates = datetime(year, 1, 1) + timedelta(float(day) - 1) + timedelta(hours = champ['Hours'][it]) 
#                     f107din = [f107d]
#                     f107ain = [f107a]
#                     time_sat = itime

#                     if str(interpolate_tiegcm(TIEGCM, lon, lat, time_sat, 400,                 'DEN')*1e3)=='-inf':
#                         print('-inf FOUND')
#                         continue
#                     elif str(interpolate_tiegcm(TIEGCM, lon, lat, time_sat, 400,                 'DEN')*1e3)=='inf':
#                         print('inf FOUND')
#                         continue
#                     else:
#                         date.append(datetime(year, 1, 1) + timedelta(days = float(day)-1,  hours = itime )) 

#                         index_date3hr = np.logical_and(mgII_data['Date_3hrAp'].year==year , mgII_data['Date_3hrAp'].dayofyear==day )
#                         indexvals =  [i for i, x in enumerate(index_date3hr) if x]
#                         Ap_doy_windows = mgII_data['Date_3hrAp'][indexvals]

#                         #### Find the Current 3hr Kp window:A
#                         Ap_windw_hrs = [i.hour for i in Ap_doy_windows]
#                         Ap_windw_hrs = np.append(np.array(Ap_windw_hrs),24)  ## add the final window edge

#                         index_current_Ap = int(np.digitize([dates.hour],Ap_windw_hrs))
#                         if index_current_Ap==8:
#                             index_current_Ap += -1
#                         indexglobal_currentAp = indexvals[index_current_Ap]

#                         Ap_3HR_current        = mgII_data['Ap'][indexglobal_currentAp]
#                         Ap_3HR_prior          = mgII_data['Ap'][indexglobal_currentAp-1]
#                         Ap_6HR_prior          = mgII_data['Ap'][indexglobal_currentAp-2]
#                         Ap_9HR_prior          = mgII_data['Ap'][indexglobal_currentAp-3]
#                         Ap_12hr_33hr_priorAVG = np.mean(mgII_data['Ap'][indexglobal_currentAp-11 :indexglobal_currentAp-3 ] ) ### 33hrs to 12 hours
#                         Ap_36hr_57hr_priorAVG = np.mean(mgII_data['Ap'][indexglobal_currentAp-19 :indexglobal_currentAp-11 ])  ### 36hrs to 57 hours

#                         apsin = [[Ap_daily_avg,          # (1) DAILY AP
#                                   Ap_3HR_current,        # (2) 3 HR AP INDEX FOR CURRENT TIME
#                                   Ap_3HR_prior,          # (3) 3 HR AP INDEX FOR 3 HRS BEFORE CURRENT TIME
#                                   Ap_6HR_prior,          # (4) 3 HR AP INDEX FOR 6 HRS BEFORE CURRENT TIME
#                                   Ap_9HR_prior,          # (5) 3 HR AP INDEX FOR 9 HRS BEFORE CURRENT TIME
#                                   Ap_12hr_33hr_priorAVG, # (6) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 12 TO 33 HRS PRIOR
#                                   Ap_36hr_57hr_priorAVG]]# (7) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 36 TO 57 HRS PRIOR

#                         output2_sat = msis.run(dates, lon, lat, champ['Height'][it], f107din, f107ain, apsin, version = 2, options=SWI_option)
#                         output2_400 = msis.run(dates, lon, lat, 400, f107din, f107ain, apsin                , version = 2, options=SWI_option)



#                         ### add the values to the growing lists
#                         rhosat_msis2[tleng+it] = output2_sat[0,0,0,0][0]
#                         rho400_msis2[tleng+it] = output2_400[0,0,0,0][0]
#                         ### RENORMALIZE with MSIS2
#                         D400_msis2[tleng+it]   = champ['Density'][it] * (output2_400[0,0,0,0][0] / output2_sat[0,0,0,0][0])   # normalized density to 400km with MSIS2

#                         ##### Interpolate the TIEGCM Data to the Satellites Position and Time
# #                         print(time_sat)
# #                         print(lon)
# #                         print(lat)
# #                         print(interpolate_tiegcm(TIEGCM, lon, lat, time_sat, champ['Height'][it], 'DEN')*1e3)
# #                         print(interpolate_tiegcm(TIEGCM, lon, lat, time_sat, 400,                 'DEN')*1e3)
#                         val_tiegcm_sat = interpolate_tiegcm(TIEGCM, lon, lat, time_sat, champ['Height'][it], 'DEN')*1e3
#                         val_tiegcm_400 = interpolate_tiegcm(TIEGCM, lon, lat, time_sat, 400,                 'DEN')*1e3
#                         rhosat_tiegcm[tleng+it] = val_tiegcm_sat
#                         rho400_tiegcm[tleng+it] = val_tiegcm_400
#                         ### RENORMALIZE with TIEGCM
# #                         print('msis2', champ['Density'][it] * (output2_400[0,0,0,0][0] / output2_sat[0,0,0,0][0]))
# #                         print('val_tiegcm_sat', val_tiegcm_sat)
# #                         print('val_tiegcm_400', val_tiegcm_400)
# #                         print('tiegcm', champ['Density'][it] * (val_tiegcm_400 / val_tiegcm_sat))
#                         D400_tiegcm[tleng+it]   = champ['Density'][it] * (val_tiegcm_400 / val_tiegcm_sat)   # normalized density to 400km with TIEGCM



#                 tleng = tleng + leng
#     #                 print(year,'/',day)
#                 i+=1
#                 print('Closing', sec_file)
#                 ncid =  Dataset(sec_file,"r+", format="NETCDF4")        
#                 ncid.close()
#                 gc.collect()


#             elif breakloop == True:
#                 i+=1
#                 continue

#     #### REMOVE NANS
#     # From Msis lists
#     rhosat_msis2 = rhosat_msis2[~np.isnan(rhosat_msis2)]
#     rho400_msis2 = rho400_msis2[~np.isnan(rho400_msis2)]
#     D400_msis2   = D400_msis2[~np.isnan(D400_msis2)]
#     # From tiegcm lists
#     rhosat_tiegcm = rhosat_tiegcm[~np.isnan(rhosat_tiegcm)]
#     rho400_tiegcm = rho400_tiegcm[~np.isnan(rho400_tiegcm)]
#     D400_tiegcm   = D400_tiegcm[~np.isnan(D400_tiegcm)]


#     df = pd.DataFrame(data={'Date' :date ,
#                             'Year'          : Year,
#                             'Doy'           : Doy,
#                             'Hours'         : Hours,
#                             'Lon'           : Lon,
#                             'Lat'           : Lat, 
#                             'LatBin'        : LatBin,
#                             'Height'        : Height,
#                             'LocTim'        : LocTim,
#                             'CHAMPDensity'  : CHAMPDensity,
#                             'rhosat_msis00' : rhosat_msis00, ### MSIS00 density @Satellite Altitude
#                             'rhosat_msis2'  : rhosat_msis2,  ### MSIS2 density @Satellite Altitude
#                             'D400_msis00'   : D400_msis00,   ### density normalized to 400km with msis00
#                             'rho400_msis2'  : rho400_msis2,  ### MSIS2 density @400km 
#                             'D400_msis2'    : D400_msis2,    ### density normalized to 400km with msis2
#                             #
#                             'rhosat_tiegcm'  : rhosat_tiegcm,  ### Tiegcm density @Satellite Altitude
#                             'rho400_tiegcm'  : rho400_tiegcm,  ### Tiegcm density @400km 
#                             'D400_tiegcm'    : D400_tiegcm,    ### density normalized to 400km with Tiegcm
#                             #                             'Cd'            : Cd,
#     #                         'Ap_dayvals'    : Ap_dayvals,
#     #                         'f107a_dayvals' : f107a_dayvals,
#     #                         'f107d_dayvals' : f107d_dayvals,
#                   } )

#     df.to_pickle('parallelize/RenormChampWithMSIS2_'+str(year))

    
#     return(df)    



# # import pandas as pd
# # import numpy as np 
# # import sys  
# # from scipy.io import loadmat  #allows us to read in .mat files
# # from datetime import datetime, timedelta

# # #### MAKE MSIS Take the 3HR Ap values
# # from pymsis import msis
# # SWI_option = [1.0]*25
# # SWI_option[8] = -1.0
# #             #  C    AP - MAGNETIC INDEX(DAILY) OR WHEN SW(9)=-1. :
# #             #  C      - ARRAY CONTAINING:
# #             #  C       (1) DAILY AP
# #             #  C       (2) 3 HR AP INDEX FOR CURRENT TIME
# #             #  C       (3) 3 HR AP INDEX FOR 3 HRS BEFORE CURRENT TIME
# #             #  C       (4) 3 HR AP INDEX FOR 6 HRS BEFORE CURRENT TIME
# #             #  C       (5) 3 HR AP INDEX FOR 9 HRS BEFORE CURRENT TIME
# #             #  C       (6) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 12 TO 33 HRS PRIOR
# #             #  C          TO CURRENT TIME
# #             #  C       (7) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 36 TO 57 HRS PRIOR
# #             #  C          TO CURRENT TIME

# # import sys  
# # sys.path.insert(0, 'util_funcs/')
# # from read_CHAMP_data import get_CHAMP_data

# # # #################################
# # # years =  [2002]#, 2003, 2004, 2005, 2006, 2007, 2008, 2009]
# # # days = np.arange(1,2)
# # # path_champ = '/space/DNR_data/CHAMP_2002_2010/'
# # # #################################


# # def make_champ_timeseries_v2_redorenorm(years, days):
    
# #     """This function redoes the construction of the CHAMP timeseries for the orbit.
    
# #     Changes from the original include:
# #         - Use F10.7 indicies that are scaled with MgII and re-referenced to the Earth's locatiion (instead of at 1AU)
# #         - Use MSIS2.0 for the normalization to 400km
# #         - Use a 3Hour Ap input as required by MSIS for the more granular Stormtime forcing option.
        
        
        
        
# #     """    

    
# #     path_champ = '/space/DNR_data/CHAMP_2002_2010/'
    
    
# #     #### Load MgII Scaled F10.7 Values:
# #     import pickle
# #     dir_save = '/space/DNR_data/'
# #     filehandler = open(dir_save+'MgII_F107_KpAp'+'.pkl', 'rb') 
# #     mgII_data = pickle.load(filehandler)
# #     filehandler.close()
        
# #     #### clear up some space        
# #     truncate_date    = np.logical_and(mgII_data['Date'].year>=2000 , mgII_data['Date'].year<=2011 )
# #     truncate_date3hr =  np.logical_and(mgII_data['Date_3hrAp'].year>=2000 , mgII_data['Date_3hrAp'].year<=2011 )#np.logical_and(mgII_data['Date_3hrAp'].year==year , mgII_data['Date_3hrAp'].dayofyear==day )

# #     mgII_data['Date_3hrAp'] = mgII_data['Date_3hrAp'][truncate_date3hr]
# #     mgII_data['Ap']         = np.array(mgII_data['Ap'])[truncate_date3hr]
# #     mgII_data['Date']       = mgII_data['Date'][truncate_date]
# #     mgII_data['Ap_dailyavg'] =  np.array(mgII_data['Ap_dailyavg'])[truncate_date]
# #     mgII_data['f107d_earth'] = np.array(mgII_data['f107d_earth'])[truncate_date]
# #     mgII_data['f107a_earth'] = np.array(mgII_data['f107a_earth'])[truncate_date]

# #     del mgII_data['DOY']
# #     del mgII_data['kp']
# #     del mgII_data['f107d']
# #     del mgII_data['f107a']
# #     del mgII_data['year_day']
        
        
        
        
# #     #     noaa = pd.read_pickle('/space/DNR_data/noaa_2002_2010_pickle')
# #     #     noaa['f107d'][noaa['f107d'].astype(float) <= 60] = noaa['f107a'][noaa['f107d'].astype(float) <= 60]

# #     tleng = 0
# #     time_full= []
# #     Year         = []
# #     Doy          = []
# #     Hours        = []
# #     Lon          = []
# #     Lat          = []
# #     LatBin       = []
# #     Height       = []
# #     LocTim       = []
# #     CHAMPDensity = []
# #     D400_msis00  = []  # normalized quantity
# #     rhosat_msis00= []
# #     rhosat_msis2 = np.ones(2000*np.size(years)*np.size(days))*np.nan
# #     rho400_msis2 = np.ones(2000*np.size(years)*np.size(days))*np.nan
# #     D400_msis2   = np.ones(2000*np.size(years)*np.size(days))*np.nan
# # #     Cd           = []
# #     # Ap_dayvals   = []
# #     # f107a_dayvals= []
# #     # f107d_dayvals= []
# #     date = []

# #     i = 0
# #     for iyear,year in enumerate(years):
# #         for iday,day in enumerate(days):
# #             champ, breakloop = get_CHAMP_data(path_champ, year, day) 
# #             if breakloop == False:
# #                 leng = np.size(champ,0)
# #     #                 date_index = datetime(year, 1, 1) + timedelta(float(day)) 
# #     #                 Ap_old    = float(noaa['Ap'][date_index])
# #     #                 f107a_old = float(noaa['f107a'][date_index])
# #     #                 f107d_old = float(noaa['f107d'][date_index])
# #                 leng2 = leng+tleng

# #                 Year[tleng:leng2]          = champ['Year'][:leng]
# #                 Doy[tleng:leng2]           = champ['Doy'][:leng]
# #                 Hours[tleng:leng2]         = champ['Hours'][:leng]
# #                 Lon[tleng:leng2]           = champ['Lon'][:leng]
# #                 Lat[tleng:leng2]           = champ['Lat'][:leng]
# #                 LatBin[tleng:leng2]        = champ['LatBin'][:leng]
# #                 Height[tleng:leng2]        = champ['Height'][:leng]
# #                 LocTim[tleng:leng2]        = champ['LocTim'][:leng]
# #                 CHAMPDensity[tleng:leng2]  = champ['Density'][:leng]
# #                 D400_msis00[tleng:leng2]   = champ['D400'][:leng]
# #                 rhosat_msis00[tleng:leng2] = champ['Dmsis'][:leng]
# # #                 Cd[tleng:leng2]            = champ['Cd'][:leng]


# #                 ####---------------------------------------------
# #                 #### Gather the Necessary Flux and Ap Information
# #                 index_date = np.logical_and(mgII_data['Date'].year==year , mgII_data['Date'].dayofyear==day )
# #                 f107a = float(np.squeeze(np.asarray(mgII_data['f107a_earth'])[index_date]))
# #                 f107d = float(np.squeeze(np.asarray(mgII_data['f107d_earth'])[index_date]))
# #                 Ap_daily_avg = float(np.squeeze(np.asarray(mgII_data['Ap_dailyavg'])[index_date]))


# #                 ### Construct the necessary 3hr Ap Array to go into MSIS
# #     #             print('---------------------------------------------')
# #                 print(f"Year: {year} / Day: {day}")
# #     #             daily_avg_ap = np.mean(Ap)
# #                 for it, itime in enumerate( champ['Hours'][:leng]):
# #                     date.append(datetime(year, 1, 1) + timedelta(days = float(day)-1,  hours = itime )) 

# #                     lon   = champ['Lon'][it]
# #                     lat   = champ['Lat'][it]
# #                     dates = datetime(year, 1, 1) + timedelta(float(day) - 1) + timedelta(hours = champ['Hours'][it]) 
# #                     f107din = [f107d]
# #                     f107ain = [f107a]

# #                     index_date3hr = np.logical_and(mgII_data['Date_3hrAp'].year==year , mgII_data['Date_3hrAp'].dayofyear==day )
# #                     indexvals =  [i for i, x in enumerate(index_date3hr) if x]
# #                     Ap_doy_windows = mgII_data['Date_3hrAp'][indexvals]

# #                     #### Find the Current 3hr Kp window:A
# #                     Ap_windw_hrs = [i.hour for i in Ap_doy_windows]
# #                     Ap_windw_hrs = np.append(np.array(Ap_windw_hrs),24)  ## add the final window edge

# #                     index_current_Ap = int(np.digitize([dates.hour],Ap_windw_hrs))
# #                     if index_current_Ap==8:
# #                         index_current_Ap += -1
# #                     indexglobal_currentAp = indexvals[index_current_Ap]

# #                     Ap_3HR_current        = mgII_data['Ap'][indexglobal_currentAp]
# #                     Ap_3HR_prior          = mgII_data['Ap'][indexglobal_currentAp-1]
# #                     Ap_6HR_prior          = mgII_data['Ap'][indexglobal_currentAp-2]
# #                     Ap_9HR_prior          = mgII_data['Ap'][indexglobal_currentAp-3]
# #                     Ap_12hr_33hr_priorAVG = np.mean(mgII_data['Ap'][indexglobal_currentAp-11 :indexglobal_currentAp-3 ] ) ### 33hrs to 12 hours
# #                     Ap_36hr_57hr_priorAVG = np.mean(mgII_data['Ap'][indexglobal_currentAp-19 :indexglobal_currentAp-11 ])  ### 36hrs to 57 hours

# #                     #### CHECK THE DATES!!!!
# #                     # print(f'      Current Date: {dates}')
# #                     # print('3HR_current ', mgII_data['Date_3hrAp'][indexglobal_currentAp])
# #                     # print('3HR_prior ', mgII_data['Date_3hrAp'][indexglobal_currentAp-1])
# #                     # print('6HR_prior ', mgII_data['Date_3hrAp'][indexglobal_currentAp-2])
# #                     # print('9HR_prior ', mgII_data['Date_3hrAp'][indexglobal_currentAp-3])
# #                     # print('12hr_33hr_priorAVG ', mgII_data['Date_3hrAp'][indexglobal_currentAp-11 :indexglobal_currentAp-3 ]  )
# #                     # print('36hr_57hr_priorAVG ', mgII_data['Date_3hrAp'][indexglobal_currentAp-19 :indexglobal_currentAp-11 ] )

# #                     apsin = [[Ap_daily_avg,          # (1) DAILY AP
# #                               Ap_3HR_current,        # (2) 3 HR AP INDEX FOR CURRENT TIME
# #                               Ap_3HR_prior,          # (3) 3 HR AP INDEX FOR 3 HRS BEFORE CURRENT TIME
# #                               Ap_6HR_prior,          # (4) 3 HR AP INDEX FOR 6 HRS BEFORE CURRENT TIME
# #                               Ap_9HR_prior,          # (5) 3 HR AP INDEX FOR 9 HRS BEFORE CURRENT TIME
# #                               Ap_12hr_33hr_priorAVG, # (6) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 12 TO 33 HRS PRIOR
# #                               Ap_36hr_57hr_priorAVG]]# (7) AVERAGE OF EIGHT 3 HR AP INDICIES FROM 36 TO 57 HRS PRIOR

# #                     output2_sat = msis.run(dates, lon, lat, champ['Height'][it], f107din, f107ain, apsin, version = 2, options=SWI_option)
# #                     output2_400 = msis.run(dates, lon, lat, 400, f107din, f107ain, apsin                , version = 2, options=SWI_option)


# #                     ### add the values to the growing lists
# #                     rhosat_msis2[tleng+it] = output2_sat[0,0,0,0][0]
# #                     rho400_msis2[tleng+it] = output2_400[0,0,0,0][0]
# #                     D400_msis2[tleng+it]   = champ['Density'][it] * (output2_400[0,0,0,0][0] / output2_sat[0,0,0,0][0])   # normalized density to 400km

# #                 tleng = tleng + leng
# #     #                 print(year,'/',day)
# #                 i+=1

# #             elif breakloop == True:
# #                 i+=1
# #                 continue

# #     rhosat_msis2 = rhosat_msis2[~np.isnan(rhosat_msis2)]
# #     rho400_msis2 = rho400_msis2[~np.isnan(rho400_msis2)]
# #     D400_msis2 = D400_msis2[~np.isnan(D400_msis2)]

# #     df = pd.DataFrame(data={'Date' :date ,
# #                             'Year'          : Year,
# #                             'Doy'           : Doy,
# #                             'Hours'         : Hours,
# #                             'Lon'           : Lon,
# #                             'Lat'           : Lat, 
# #                             'LatBin'        : LatBin,
# #                             'Height'        : Height,
# #                             'LocTim'        : LocTim,
# #                             'CHAMPDensity'  : CHAMPDensity,
# #                             'rhosat_msis00' : rhosat_msis00, ### MSIS00 density @Satellite Altitude
# #                             'rhosat_msis2'  : rhosat_msis2,  ### MSIS2 density @Satellite Altitude
# #                             'D400_msis00'   : D400_msis00,   ### density normalized to 400km with msis00
# #                             'rho400_msis2'  : rho400_msis2,  ### MSIS2 density @400km 
# #                             'D400_msis2'    : D400_msis2,    ### density normalized to 400km with msis2
# # #                             'Cd'            : Cd,
# #     #                         'Ap_dayvals'    : Ap_dayvals,
# #     #                         'f107a_dayvals' : f107a_dayvals,
# #     #                         'f107d_dayvals' : f107d_dayvals,
# #                   } )

# #     df.to_pickle('parallelize/RenormChampWithMSIS2_'+str(year))
# # #     df.to_pickle('constructed_files/RenormChampWithMSIS2_'+year)

    
# #     return    
